In [2]:
#import opendatasets as od
#od.download("https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification")
#import nltk
#nltk.download('wordnet') 
#nltk.download('omw-1.4')
#skip the blocks #27-#32 and #90 

import re #regex
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from nltk.tokenize import TweetTokenizer #tokenizer for tweets, documentation is found here: https://www.nltk.org/api/nltk.tokenize.casual.html
from nltk.stem import WordNetLemmatizer #lemmatizes words, documentation here: https://www.nltk.org/_modules/nltk/stem/wordnet.html
from nltk.corpus import stopwords #this is used to remove any stopwords / filler words
from nltk.stem import PorterStemmer #stemmer
from sklearn.feature_extraction.text import CountVectorizer #matrix of token counts: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import TfidfVectorizer #equivalent to CountVectorizer then TfidfTransformer: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
from sklearn.model_selection import train_test_split #split arrays/matrices into random train and test subsets: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.ensemble import RandomForestClassifier #this is not needed since it is the section we are skipping
from sklearn.model_selection import cross_val_score #this isn't used in the og code, so idk what they intended to use this for
    #however it could be used to get a score by cross validation
from sklearn.metrics import confusion_matrix #this allows me to create a confusion matrix
#import xgboost 
    #I keep getting an error that says "ModuleNotFoundError: No module named 'xgboost'"
    #I kept getting this error after installing it in my conda enviroment
    #When trying to use "pip install xgboost" to install this module, I kept getting an error that starts with "ERROR: Command errored out with exit status 1:"
    #followed by a VERY long error message followed by 
        #ERROR: Failed building wheel for xgboost
        #Running setup.py clean for xgboost
        #Failed to build xgboost
        #Installing collected packages: xgboost
        #Running setup.py install for xgboost ... error
    #and then it is followed by "error: [Errno 2] No such file or directory: 'cmake'"
from sklearn.model_selection import RandomizedSearchCV #

In [3]:
train=pd.read_csv("./covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding='latin1')
#changed the path from "/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv"
#we are using pandas to read the csv
#the encoding allows us to read latin characters (e.g. ê, æ, etc)
#found this website that shows whats in Latin1: https://kb.iu.edu/d/aepu 

In [4]:
train.head() #used to see the first 5 rows of the .csv to check if it is the right one

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [5]:
def drop(p):
    p.drop(["UserName","ScreenName","Location","TweetAt"],axis=1,inplace=True)
    #this function removes the UserName, ScreenName, Location, and TweetAt columns and keeps the rest of the columns in order
    #documentation can be found here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html

In [6]:
drop(train) #use the drop function on the train set

In [7]:
train.head() #check mutated train set

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [8]:
tweettoken = TweetTokenizer(strip_handles=True, reduce_len=True) #tokenizes the tweets
#strip_handles removes handles and reduce_len reduces the length of 3+ repeated characters  

In [9]:
lemmatizer=WordNetLemmatizer() #lemmatizer for word net words: https://www.nltk.org/_modules/nltk/stem/wordnet.html

In [10]:
stemmer=PorterStemmer() #the porter stemmer: https://www.nltk.org/_modules/nltk/stem/porter.html

In [11]:
collect=[] #list to store the preprocess function output thing
def preprocess(t):
    tee=re.sub('[^a-zA-Z]'," ",t) #replace anything that begins with a alpha character with a whitespace: https://docs.python.org/3/library/re.html
    tee=tee.lower() #make it all lowercase
    res=tweettoken.tokenize(tee) #tokenize the tweets --> becomes a list
    for i in res:
        if i in stopwords.words('english'): #remove the stopwords
            res.remove(i)
    rest=[]
    for k in res:
        rest.append(lemmatizer.lemmatize(k)) #lemmatize the res list and shove it into the rest list
    ret=" ".join(rest) #join rest together with whitespaces in between
    collect.append(ret) #put this all into the the collect list outside of this function

In [12]:
for j in range(41157): #b the range is literally 41k, it takes forever (a couple of minutes) to do this function
    preprocess(train["OriginalTweet"].iloc[j])
    #needed to download nltk.wordnet using nltk.download('wordnet') and nltk.download('omw-1.4')

In [13]:
collect[:5] #the :5 shows the first 5 in the list

['menyrbie phil gahan chrisitv http co ifz fan pa http co xx ghgfzcc http co nlzdxno',
 'advice talk your neighbour family exchange phone number create contact list phone number neighbour school employer chemist gp set online shopping account po adequate supply regular med not order',
 'coronavirus australia woolworth give elderly disabled dedicated shopping hour amid covid outbreak http co binca vp p',
 'food stock not only one is empty please panic will enough food everyone not take than you need stay calm stay safe covid france covid covid coronavirus confinement confinementotal confinementgeneral http t co zrlg z j',
 'ready go supermarket covid outbreak because m paranoid because food stock litteraly empty the coronavirus a serious thing please panic cause shortage coronavirusfrance restezchezvous stayathome confinement http t co usmualq n']

In [14]:
def bow(ll): #parameter is double lowercase l, maybe for "lemmatized list" or smth
    cv=CountVectorizer(max_features=200) #limit the max features of the vector to 200
    x=cv.fit_transform(ll).toarray() #both the .fit and .transform functions together, returns a matrix --> .toarray() makes it an array
    #documentation for fit_transform: https://github.com/scikit-learn/scikit-learn/blob/36958fb24/sklearn/feature_extraction/text.py#L1294
    return x

In [15]:
y=bow(collect) #using the bow function on the collect list and setting it to y

In [16]:
y[:1] #this shows the first vector of y

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]])

In [17]:
len(y[0][:]) #this shows the length of the first vector

200

In [18]:
def tfidf(xx): ##tfidf refers to TF-IDF features which is "term frequency-inverse document frequency"
    #statistical measure that evaluates how relevant a word is to a document in a collection of documents
    cv=TfidfVectorizer(max_features=4000) #max features set to 4000
    x=cv.fit_transform(xx).toarray() #similar to what happened with the bow function
    return x

In [19]:
values=train["Sentiment"].values #get the values of the sentiment column

In [20]:
values #the implementation we are following has the results as "array([2, 0, 0, ..., 0, 2, 1])"
#but the values that I get here are the literal strings

array(['Neutral', 'Positive', 'Positive', ..., 'Positive', 'Neutral',
       'Negative'], dtype=object)

In [21]:
print(y.shape)
print(values.shape)
#these were to double check the lengths of these arrays
#since at first I got an error when trying to use train_test_split bc the size of y and values did not match

(41157, 200)
(41157,)


In [22]:
(x_train, x_test, y_train, y_test) = train_test_split(y, values, train_size=0.75, random_state=42) #make random train and test sets
#double checked the lengths of both y and values using .shape
#documentation was found here: https://numpy.org/devdocs/reference/generated/numpy.shape.html

In [23]:
x_train #look at the x values of the train set

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
from sklearn.naive_bayes import MultinomialNB #multinomial Naïve Bayes
clf = MultinomialNB()
clf.fit(x_train,y_train)  #fit the naive bayes classifier with the training sets

MultinomialNB()

In [25]:
clf.score(x_test,y_test) #need to compare this value with the other score

0.3825072886297376

In [31]:
#I followed this stackoverflow answer to figure out how to print out a confusion matrix for Naïve Bayes
#https://stackoverflow.com/questions/48412360/how-can-i-use-confusion-matrix-with-naive-bayes-in-python
y_pred = clf.predict(x_test) #so we can get the predictions with the x test set
con_mat = confusion_matrix(y_test, y_pred) #compare the actual test with the predictions
print(con_mat) #this is the confusion matrix

[[ 435   58  384  200  240]
 [  71  670  153  233  516]
 [ 380  177  769  601  586]
 [  65   62  325 1061  406]
 [ 174  502  512  709 1001]]


In [32]:
y=tfidf(collect) #use the tfidf function on the collect list

In [33]:
(x_train,x_test,y_train,y_test) = train_test_split(y,values, train_size=0.75, random_state=42) #make random train and test sets

In [34]:
clf = MultinomialNB()
clf.fit(x_train,y_train) #same as before
clf.score(x_test,y_test) #need to compare this score with the previous one

0.46763848396501456

In [35]:
y_pred = clf.predict(x_test) #I basically just copy-pasted the same code I used for the first confusion matrix
con_mat = confusion_matrix(y_test, y_pred)
print(con_mat) #this is the confusion matrix

[[ 262    2  807   32  214]
 [   2  415   78   34 1114]
 [  94   26 1295  186  912]
 [   8   20  350  774  767]
 [  15  131  491  195 2066]]


In [ ]:
#1. The output the .score() function provides is the accuracy of the current classifier.
#I found a good explanation about .score() at https://www.kaggle.com/getting-started/27261

#2. The reason why that the second score is bigger than the first score is because CountVectorizer is just counting the frequency of the words.
# TfidfVectorizer however does not only count the frequency of the words, but also assigns importance (tfidf score) to each word.
#e.g. If a word was used too rarely, it would be given low tfidf score. This affects the overall score when computed.
#I found this comparison in this blog post: https://www.linkedin.com/pulse/count-vectorizers-vs-tfidf-natural-language-processing-sheel-saket/